# Data Preprocessing
### Resident population & Float population

In [1]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm

In [2]:
float_people = pd.read_excel('data/float_people.xlsx')
resid_people = pd.read_excel('data/resid_people.xls')

### - Concatening resident population columns

In [62]:
resid_people.iloc[:, ]

,Unnamed: 0,0~4세,5~9세,10~14세,15~19세,20~24세,25~29세,30~34세,35~39세,40~44세,...,55~59세.1,60~64세.1,65~69세.1,70~74세.1,75~79세,80~84세,85~89세,90~94세,95~99세,100세 이상
0,대전광역시 (3000000000),26663,35006,35993,42328,54714,56962,48353,54697,55196,...,59587,52319,35651,26283,22287,16294,9049,3493,829,259
1,대전광역시 동구 (3011000000),3545,4663,4874,5795,8431,7848,6421,7961,8229,...,9376,9362,6930,5456,4837,3430,1734,664,156,63
2,대전광역시 동구 중앙동(3011051500),19,33,26,57,95,126,116,144,156,...,222,268,255,226,199,146,65,25,10,4
3,대전광역시 동구 효동(3011053000),539,867,919,869,803,616,629,981,1183,...,909,912,627,524,385,272,149,61,13,5
4,대전광역시 동구 신인동(3011054500),248,251,232,265,371,389,436,506,472,...,523,572,435,344,289,198,85,47,5,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,대전광역시 대덕구 석봉동(3023056000),411,514,326,359,476,495,561,714,626,...,550,455,265,165,166,128,70,23,4,1
81,대전광역시 대덕구 덕암동(3023057000),193,262,316,402,589,611,446,494,504,...,581,490,363,269,260,171,98,34,9,2
82,대전광역시 대덕구 목상동(3023058000),116,161,163,172,287,346,325,334,321,...,281,195,111,83,81,59,27,11,6,1
83,대전광역시 대덕구 법1동(3023060000),98,149,210,241,403,316,236,252,277,...,496,480,374,325,251,205,131,23,11,1


In [66]:
def concat_col(df, n):
    return df.iloc[:, n] + df.iloc[:, n+1]
    

In [67]:
resid_people.columns

Index(['Unnamed: 0', '0~4세', '5~9세', '10~14세', '15~19세', '20~24세', '25~29세',
       '30~34세', '35~39세', '40~44세', '45~49세', '50~54세', '55~59세', '60~64세',
       '65~69세', '70~74세', '0~4세.1', '5~9세.1', '10~14세.1', '15~19세.1',
       '20~24세.1', '25~29세.1', '30~34세.1', '35~39세.1', '40~44세.1', '45~49세.1',
       '50~54세.1', '55~59세.1', '60~64세.1', '65~69세.1', '70~74세.1', '75~79세',
       '80~84세', '85~89세', '90~94세', '95~99세', '100세 이상'],
      dtype='object')

In [70]:
total = 0
for i in (range(7)):
    n = 2*i+1
    resid_people.iloc[:, n] = (concat_col(resid_people, n))
    
for i in (range(7,18)):
    n = 2*i+1
    resid_people.iloc[:, n] = (concat_col(resid_people, n))

for i in range(30,37):
    total += resid_people.iloc[:, i]
    resid_people.iloc[:, 30] = total

1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35


In [71]:
drop_index = list(range(2,15,2))+list(range(17,30,2))+list(range(31,37))
resid_people.drop(columns=list(resid_people.columns[drop_index]), inplace=True)

### - Process as a proper type

In [74]:
float_people.columns

Index(['연월', '시', '구', '행정동', '시간대', ' 남10대 미만 ', ' 남10대 ', ' 남20대 ', ' 남30대 ',
       ' 남40대 ', ' 남50대 ', ' 남60대 ', ' 남70대 이상 ', ' 여10대 미만 ', ' 여10대 ',
       ' 여20대 ', ' 여30대 ', ' 여40대 ', ' 여50대 ', ' 여60대 ', ' 여70대 이상 '],
      dtype='object')

In [77]:
float_col = list(float_people.columns[5:])
resid_col = list(resid_people.columns[1:])
len(float_col) == len(resid_col)

True

In [78]:
col_change = {}
for i in range(len(float_col)):
    col_change[resid_col[i]] = float_col[i]

In [82]:
resid_people.rename(columns=col_change, inplace=True)

In [100]:
resid_people.rename(columns={'Unnamed: 0': '시'}, inplace=True)

In [87]:
resid_people.insert(1, '행정동', 0)
resid_people.insert(1, '구', 0)

In [105]:
df = resid_people.copy()

In [106]:
drop_row = []
for i in range(len(df)):
    if re.search('.+구 .+동', df.iloc[i,0]) == None:
        drop_row.append(i)
    else:
        df.iloc[i,1] = re.findall('대전광역시 (.+구) .+동', df.iloc[i,0])[0]
        df.iloc[i,2] = re.findall('대전광역시 .+구 (.+동)', df.iloc[i,0])[0]
        df.iloc[i,0] = '대전광역시'
        
df.drop(index = drop_row, inplace=True)

In [107]:
df

,시,구,행정동,남10대 미만,남10대,남20대,남30대,남40대,남50대,남60대,남70대 이상,여10대 미만,여10대,여20대,여30대,여40대,여50대,여60대,여70대 이상
2,대전광역시,동구,중앙동,85,83,221,260,376,689,641,256,27,37,75,83,88,162,268,850
3,대전광역시,동구,효동,2273,1788,1419,1610,2526,1824,1374,988,548,839,681,623,1268,1007,912,1747
4,대전광역시,동구,신인동,750,497,760,942,1000,958,794,577,267,208,313,370,407,504,572,1231
5,대전광역시,동구,판암1동,710,542,819,850,880,1085,887,477,210,216,356,347,389,595,578,1341
6,대전광역시,동구,판암2동,248,221,528,451,589,808,885,336,60,96,183,145,185,347,533,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,대전광역시,대덕구,석봉동,1439,685,971,1275,1225,1166,733,556,396,314,358,528,533,611,455,709
81,대전광역시,대덕구,덕암동,717,718,1200,940,1232,1446,841,437,192,270,452,332,469,655,490,1050
82,대전광역시,대덕구,목상동,438,335,633,659,678,647,350,195,113,162,230,203,250,256,195,339
83,대전광역시,대덕구,법1동,396,451,719,488,659,952,709,287,94,198,326,204,282,524,480,1176


In [109]:
df.to_excel('resid_people_revised.xls')

In [3]:
float_people.shape

(28440, 21)

In [4]:
df = pd.DataFrame(data=np.zeros([455040,9]), columns=['시', '구', '행정동', '년', '월', '시간대', '성별', '연령대', '유동인구'])

In [5]:
df['시'] = '대전광역시'

In [6]:
index = 0
age = ['10대 미만', '10대', '20대', '30대', '40대', '50대', '60대', '70대 이상']
for i in tqdm(range(len(float_people))):
        df.loc[index:index+15, '구'] = float_people.loc[i, '구']
        df.loc[index:index+15, '행정동'] = float_people.loc[i, '행정동']
        df.loc[index:index+15, '년'] = str(float_people.loc[i, '연월'])[:4]
        df.loc[index:index+15, '월'] = str(float_people.loc[i, '연월'])[4:]
        df.loc[index:index+15, '시간대'] = float_people.loc[i, '시간대']
        
        df.loc[index:index+7, '성별'] = '남'
        df.loc[index+8:index+15, '성별'] = '여'
        df.loc[index:index+7, '연령대'] = age
        df.loc[index+8:index+15, '연령대'] = age
        
        df.loc[index:index+7, '유동인구'] = float_people.iloc[i, 5:13].values
        df.loc[index+8:index+15, '유동인구'] = float_people.iloc[i, 13:].values
        index+=16

100%|██████████| 28440/28440 [1:46:48<00:00,  4.44it/s]  


In [26]:
df.to_csv('float_people_revised.csv', encoding='utf8')

In [42]:
resid = pd.read_excel('resid_people_revised.xls', index_col=0)

In [43]:
resid.reset_index(inplace=True)

In [46]:
df2 = pd.DataFrame(data = np.zeros([1264, 6]), columns=['시', '구', '행정동', '성별', '연령대', '거주인구'])
index = 0
age = ['10대 미만', '10대', '20대', '30대', '40대', '50대', '60대', '70대 이상']
df2['시'] = '대전광역시'
for i in tqdm(range(len(resid))):
        df2.loc[index:index+15, '구'] = resid.loc[i, '구']
        df2.loc[index:index+15, '행정동'] = resid.loc[i, '행정동']

        df2.loc[index:index+7, '성별'] = '남'
        df2.loc[index+8:index+15, '성별'] = '여'
        df2.loc[index:index+7, '연령대'] = age
        df2.loc[index+8:index+15, '연령대'] = age

        df2.loc[index:index+7, '거주인구'] = resid.iloc[i, 4:12].values
        df2.loc[index+8:index+15, '거주인구'] = resid.iloc[i, 12:].values
        index+=16

100%|██████████| 79/79 [00:00<00:00, 364.17it/s]


In [47]:
df2

,시,구,행정동,성별,연령대,거주인구
0,대전광역시,동구,중앙동,남,10대 미만,85
1,대전광역시,동구,중앙동,남,10대,83
2,대전광역시,동구,중앙동,남,20대,221
3,대전광역시,동구,중앙동,남,30대,260
4,대전광역시,동구,중앙동,남,40대,376
...,...,...,...,...,...,...
1259,대전광역시,대덕구,법2동,여,30대,344
1260,대전광역시,대덕구,법2동,여,40대,572
1261,대전광역시,대덕구,법2동,여,50대,889
1262,대전광역시,대덕구,법2동,여,60대,745


In [48]:
df2.to_csv('resid_people_revised.csv')